# Loading and Adjusting Sharadar US Equity Prices

The notebook demonstrates and tests the get_pricing method for loading and adjusting the Sharadar SEP bundle (ingested with the sep_ingest notebook in this repository). The corresponding example consists of three steps: (1) getting adjusted data with get_pricing from Zipline, (2) reading split-adjusted data from Sharadar files, and (3) adding dividend adjustments to Sharadar to test with data generated by the get_pricing method.

In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Importing the fsharadar.sep module from the Flounder extension

In [2]:
from fsharadar import sep

## 1. Getting Historical Prices Adjusted for Splits and Dividends

In [3]:
start_date = "2011-01-03"; end_date = "2021-02-12"

Selecting one field from Sharadar SEP bundle

In [4]:
sep.bundle_tags

['close', 'high', 'low', 'open', 'volume']

In [5]:
field = 'close'

Getting all tickers maintained in Sharadar SEP bundle

In [6]:
sep_bundle_data = sep.load()
sep_asset_finder = sep_bundle_data.asset_finder
sep_assets = sep_asset_finder.retrieve_all(sep_asset_finder.sids)
sep_tickers = [asset.symbol for asset in sep_assets]
len(sep_tickers)

11667

Applying get_pricing for loading adjusted stock prices

In [7]:
%%time
prices = sep.get_pricing(sep_tickers, start_date, end_date, field)

CPU times: user 21.7 s, sys: 1.36 s, total: 23.1 s
Wall time: 22.6 s


In [8]:
prices.head(2)

,Equity(101386 [GBBT]),Equity(101501 [BBUCQ]),Equity(101512 [GOVB]),Equity(101923 [CIBN]),Equity(103609 [AMCRY]),Equity(103618 [DASTF]),Equity(103628 [AXAHY]),Equity(103638 [NHYDY]),Equity(103642 [KKPNY]),Equity(103688 [BASFY]),...,Equity(633630 [CTAQU]),Equity(633631 [DWIN.U]),Equity(633632 [HMCOU]),Equity(633633 [ALTUU]),Equity(633634 [MUDSU]),Equity(633635 [NOACU]),Equity(633636 [SVFAU]),Equity(633638 [IPOC.U]),Equity(633639 [GSAH.U]),Equity(633851 [FSII])
2011-01-03 00:00:00+00:00,NaN,0.01,6.787,4.063,15.313,NaN,10.385,5.366,5.035,13.596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.41
2011-01-04 00:00:00+00:00,NaN,0.01,6.787,4.063,15.168,NaN,10.574,5.266,5.055,13.246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.17


## 2. Loading Sharadar files with split-adjusted prices, dividends, and volumes

Tickers (with sids)

In [9]:
sharadar_tickers_file = "./SHARADAR_TICKERS.csv"

In [10]:
tickers_df = pd.read_csv(sharadar_tickers_file)
print(len(tickers_df.ticker.unique()))

23794


SEP (Sharadar Equity Prices) 

In [11]:
sharadar_sep_file = "./SHARADAR_SEP.csv"

In [12]:
%%time
from fsharadar.sep.ingest import read_sep_file

sep_df = read_sep_file(sharadar_sep_file, tickers_df)
print(len(sep_df.symbol.unique()))

11667
CPU times: user 17.7 s, sys: 1.05 s, total: 18.7 s
Wall time: 18.7 s


In [13]:
raw_data = sep_df.set_index(['symbol', 'date'])
raw_data.sort_index(level=0, inplace=True)

In [14]:
raw_data.head(2)

open   high     low  close     volume  dividends
symbol date                                                         
A      2011-01-03  41.56  42.14  41.411  41.88  3572300.0        0.0
       2011-01-04  41.99  42.10  41.180  41.49  3588900.0        0.0

## 3. Checking Differences in Adjusted Prices

The following code runs through each ticker, adds dividend adjustments to Sharadar split-adjusted prices, and checks them for potential discrepencies with the corresponding adjusted prices from the get_pricing method of the fsharadar.sep module. 

 Dividend adjustments are calculated via a (modified) Python function outlined in Josh Schertz's blog post:
 https://joshschertz.com/2016/08/27/Vectorizing-Adjusted-Close-with-Python/

In [15]:
def calculate_adjusted_prices(df, column):

    # Reverse the DataFrame order, sorting by date in descending order
    df.sort_index(ascending=False, inplace=True)

    prices = df[column].values
    dividends = df['dividends'].values
    
    adj_prices = np.zeros(len(df.index))
    adj_prices[0] = prices[0]

    for i in range(1, len(prices)):    
        delta = (prices[i] - dividends[i - 1]) / prices[i - 1]
        adj_prices[i] = adj_prices[i - 1] * delta
        
    adj_ts = pd.Series(adj_prices, index=df.index)

    # Change the DataFrame order back to dates ascending
    adj_ts.sort_index(ascending=True, inplace=True)

    return adj_ts

In [16]:
%%time

for i, asset in enumerate(prices.columns):
    
    ticker = asset.symbol
    raw_xs = raw_data.xs(ticker)   
    adj_raw_ts = calculate_adjusted_prices(raw_xs, field).dropna()
    
    asset_prices_ts = prices[asset].dropna() 
    
    std_diff = np.std(adj_raw_ts.values - asset_prices_ts.values)
    
    if std_diff > 5.e-4:
        print(i, ticker, std_diff)  

CPU times: user 22 s, sys: 23.6 ms, total: 22.1 s
Wall time: 22.1 s
